In [1]:
# Import libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [2]:
# Set image size and batch size
img_size = (128, 128)
batch_size = 32

In [3]:
# Prepare training and validation data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,       # normalize pixel values
#    shear_range=0.2,
#    zoom_range=0.2,
#    horizontal_flip=True,
    validation_split=0.2  # 20% for validation
)

In [4]:

train_generator = train_datagen.flow_from_directory(
    'E:\\dataset\\training_set',   # path to dataset folder
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

Found 6400 images belonging to 2 classes.


In [5]:
val_generator = train_datagen.flow_from_directory(
    'E:\\dataset\\training_set',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

Found 1600 images belonging to 2 classes.


In [6]:
# Build CNN model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')   # Binary classification (dog or cat)
])

c:\Users\Sajid Ali\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

Epoch 1/10


c:\Users\Sajid Ali\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


200/200 ━━━━━━━━━━━━━━━━━━━━ 396s 2s/step - accuracy: 0.5604 - loss: 0.7745 - val_accuracy: 0.6469 - val_loss: 0.6604
Epoch 2/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 229s 1s/step - accuracy: 0.6573 - loss: 0.6307 - val_accuracy: 0.7150 - val_loss: 0.5760
Epoch 3/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 85s 424ms/step - accuracy: 0.7413 - loss: 0.5181 - val_accuracy: 0.7119 - val_loss: 0.5525
Epoch 4/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 82s 405ms/step - accuracy: 0.8088 - loss: 0.4236 - val_accuracy: 0.7437 - val_loss: 0.5345
Epoch 5/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 100s 499ms/step - accuracy: 0.8658 - loss: 0.3227 - val_accuracy: 0.7281 - val_loss: 0.5967
Epoch 6/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 90s 433ms/step - accuracy: 0.9131 - loss: 0.2130 - val_accuracy: 0.7506 - val_loss: 0.6196
Epoch 7/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 70s 347ms/step - accuracy: 0.9486 - loss: 0.1404 - val_accuracy: 0.7425 - val_loss: 0.7641
Epoch 8/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 70s 345ms/step - accuracy: 0.9692 - loss: 0.0891 - val_ac

In [8]:
# 4️⃣ Evaluate Model
# ----------------------------
loss, acc = model.evaluate(val_generator)
print(f"Validation Accuracy: {acc*100:.2f}%")

# Save model
model.save('dog_cat_cnn_model.h5')


50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 101ms/step - accuracy: 0.7496 - loss: 1.0385


Validation Accuracy: 75.06%


In [10]:
# 5️⃣ Prediction on Test Dataset
test_path = 'E:\\dataset\\test_set'

# Get class indices for mapping (e.g., {'cats': 0, 'dogs': 1})
class_indices = train_generator.class_indices
print("Class indices:", class_indices)

Class indices: {'cats': 0, 'dogs': 1}


In [11]:
# Define function for single image prediction
def predict_image(image_path):
    img = load_img(image_path, target_size=img_size)
    img_array = img_to_array(img) / 255.0   # normalize
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)
    
    label = "🐶 Dog" if prediction[0][0] > 0.5 else "🐱 Cat"
    print(f"{os.path.basename(image_path)} → {label}")

# ----------------------------
# Predict only first 5 test images
# ----------------------------
test_images = []

# Collect image paths from all subfolders
for folder in os.listdir(test_path):
    folder_path = os.path.join(test_path, folder)
    for file in os.listdir(folder_path):
        img_path = os.path.join(folder_path, file)
        test_images.append(img_path)

# Select first 5 images
test_images = test_images[40:50]

# Predict on these images
for img_path in test_images:
    predict_image(img_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step
cat.4041.jpg → 🐱 Cat
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
cat.4042.jpg → 🐱 Cat
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
cat.4043.jpg → 🐱 Cat
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
cat.4044.jpg → 🐶 Dog
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
cat.4045.jpg → 🐱 Cat
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
cat.4046.jpg → 🐱 Cat
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
cat.4047.jpg → 🐶 Dog
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
cat.4048.jpg → 🐱 Cat
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
cat.4049.jpg → 🐱 Cat
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
cat.4050.jpg → 🐱 Cat


In [ ]:
import random

test_images = []
for folder in os.listdir(test_path):
    folder_path = os.path.join(test_path, folder)
    for file in os.listdir(folder_path):
        img_path = os.path.join(folder_path, file)
        test_images.append(img_path)

# Shuffle and take first 10 images
random.shuffle(test_images)
test_images = test_images[:10]

for img_path in test_images:
    predict_image(img_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
dog.4695.jpg → 🐶 Dog
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
dog.4735.jpg → 🐶 Dog
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
dog.4268.jpg → 🐶 Dog
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
cat.4761.jpg → 🐱 Cat
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
dog.4756.jpg → 🐶 Dog
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
dog.4585.jpg → 🐶 Dog
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
cat.4962.jpg → 🐱 Cat
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
cat.4381.jpg → 🐱 Cat
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
dog.4880.jpg → 🐱 Cat
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
dog.4751.jpg → 🐶 Dog
